# Traitement de la base sirene

In [5]:
import os

import pandas as pd

In [2]:
# Ne pas oublier PEP 8

In [62]:
# Déclaration des constantes
DATA_DIR = '../data/external/'
ENTITIES_FILENAME = 'StockUniteLegale_utf8.zip'
ETAB_FILENAME = 'StockEtablissement_utf8.zip'

ACTIVITY_CODE = '86.'

## Préparation des entités

In [6]:
pd.read_csv(os.path.join(DATA_DIR, ENTITIES_FILENAME), # Concaténation du répertoire et du nom de fichier
            compression='zip',
            #nrows=1000 # limite à 1000 lignes la lecture
           ).head()

,siren,statutdiffusionunitelegale,unitepurgeeunitelegale,datecreationunitelegale,sigleunitelegale,sexeunitelegale,prenom1unitelegale,prenom2unitelegale,prenom3unitelegale,prenom4unitelegale,...,denominationunitelegale,denominationusuelle1unitelegale,denominationusuelle2unitelegale,denominationusuelle3unitelegale,categoriejuridiqueunitelegale,activiteprincipaleunitelegale,nomenclatureactiviteprincipaleunitelegale,nicsiegeunitelegale,economiesocialesolidaireunitelegale,caractereemployeurunitelegale
0,325175,O,NaN,2000-09-26,NaN,M,THIERRY,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1000,32.12Z,NAFRev2,65,NaN,N
1,1807254,O,NaN,1972-05-01,NaN,M,JACQUES-LUCIEN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1000,85.59A,NAFRev2,22,NaN,N
2,5410220,O,True,1954-12-25,NaN,M,GEORGES,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1000,22.02,NAP,15,NaN,N
3,5410345,O,True,NaN,NaN,M,MICHEL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1000,79.06,NAP,10,NaN,N
4,5410394,O,True,1954-12-25,NaN,M,ROBERT,ALFRED,NaN,NaN,...,NaN,NaN,NaN,NaN,1000,64.42,NAP,18,NaN,N


In [8]:
pd.read_csv(os.path.join(DATA_DIR, ENTITIES_FILENAME), # Concaténation du répertoire et du nom de fichier
            compression='zip',
            nrows=1000 # limite à 1000 lignes la lecture
           ).columns

Index(['siren', 'statutdiffusionunitelegale', 'unitepurgeeunitelegale',
       'datecreationunitelegale', 'sigleunitelegale', 'sexeunitelegale',
       'prenom1unitelegale', 'prenom2unitelegale', 'prenom3unitelegale',
       'prenom4unitelegale', 'prenomusuelunitelegale', 'pseudonymeunitelegale',
       'identifiantassociationunitelegale', 'trancheeffectifsunitelegale',
       'anneeeffectifsunitelegale', 'datederniertraitementunitelegale',
       'nombreperiodesunitelegale', 'categorieentreprise',
       'anneecategorieentreprise', 'datedebut', 'etatadministratifunitelegale',
       'nomunitelegale', 'nomusageunitelegale', 'denominationunitelegale',
       'denominationusuelle1unitelegale', 'denominationusuelle2unitelegale',
       'denominationusuelle3unitelegale', 'categoriejuridiqueunitelegale',
       'activiteprincipaleunitelegale',
       'nomenclatureactiviteprincipaleunitelegale', 'nicsiegeunitelegale',
       'economiesocialesolidaireunitelegale', 'caractereemployeurunitelega

In [95]:
%%time
entities_df = (pd.read_csv(os.path.join(DATA_DIR, ENTITIES_FILENAME), # Concaténation du répertoire et du nom de fichier
                           usecols=['siren',
                                    'activiteprincipaleunitelegale', 'datecreationunitelegale',
                                    'denominationunitelegale', 'nomunitelegale', 'etatadministratifunitelegale',
                                   ],
                           compression='zip',
                           #nrows=100000 # limite à n lignes la lecture
                          )
             ).dropna(subset=['activiteprincipaleunitelegale']).query('etatadministratifunitelegale == "A"')

# Filtrage des professionnels de santé
#entities_df = entities_df[entities_df.activiteprincipaleunitelegale.str.startswith(ACTIVITY_CODE)] # 86.

<string>:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 44.9 s


In [96]:
entities_df.shape

(11696143, 6)

In [57]:
entities_df.denominationunitelegale.isnull().sum()

469609

In [58]:
entities_df.nomunitelegale.isnull().sum()

48443

In [59]:
entities_df.denominationunitelegale.isnull().sum() + entities_df.nomunitelegale.isnull().sum()

518052

In [60]:
entities_df.head()

,datecreationunitelegale,etatadministratifunitelegale,nomunitelegale,denominationunitelegale,activiteprincipaleunitelegale
801,1957-01-01,A,NaN,SA SAINTE-ISABELLE,86.10Z
4081,1965-01-01,A,MORONI,NaN,86.90A
5775,1991-06-03,A,DACQUIN,NaN,86.06
5893,1970-01-01,A,NaN,CENTRE DES CARMES,86.10Z
15153,1971-01-01,A,NaN,CLINIQUE LES ROSIERS,86.10Z


## Préparation des établissements

In [63]:
pd.read_csv(os.path.join(DATA_DIR, ETAB_FILENAME), # Concaténation du répertoire et du nom de fichier
            compression='zip',
            nrows=1000 # limite à 1000 lignes la lecture
           ).head()

,siren,nic,siret,statutdiffusionetablissement,datecreationetablissement,trancheeffectifsetablissement,anneeeffectifsetablissement,activiteprincipaleregistremetiersetablissement,datederniertraitementetablissement,etablissementsiege,...,libellepaysetranger2etablissement,datedebut,etatadministratifetablissement,enseigne1etablissement,enseigne2etablissement,enseigne3etablissement,denominationusuelleetablissement,activiteprincipaleetablissement,nomenclatureactiviteprincipaleetablissement,caractereemployeuretablissement
0,325175,16,32517500016,O,2000-09-26,NaN,NaN,3212ZZ,2015-03-18T00:58:59,False,...,NaN,2009-05-27,F,NaN,NaN,NaN,NaN,32.12Z,NAFRev2,N
1,325175,24,32517500024,O,2008-05-20,NaN,NaN,NaN,2011-12-12T09:40:04,False,...,NaN,2011-10-21,F,TAHITI PERLES CREATIONS,NaN,NaN,NaN,47.89Z,NAFRev2,N
2,325175,32,32517500032,O,2009-05-27,NaN,NaN,NaN,2014-07-08T00:10:21,False,...,NaN,2011-10-21,F,TAHITI PERLES CREATIONS,NaN,NaN,NaN,32.12Z,NAFRev2,N
3,325175,40,32517500040,O,2011-10-21,NaN,NaN,3212ZZ,2015-03-18T00:58:59,False,...,NaN,2014-01-07,F,TAHITI PERLES CREATION,NaN,NaN,NaN,32.12Z,NAFRev2,N
4,325175,57,32517500057,O,2014-01-07,NaN,NaN,NaN,2018-07-10T14:17:15,False,...,NaN,2018-02-07,F,TAHITI PERLES CREATION,NaN,NaN,NaN,32.12Z,NAFRev2,N


In [65]:
pd.read_csv(os.path.join(DATA_DIR, ETAB_FILENAME), # Concaténation du répertoire et du nom de fichier
            compression='zip',
            nrows=1000 # limite à 1000 lignes la lecture
           ).columns.sort_values()

Index(['activiteprincipaleetablissement',
       'activiteprincipaleregistremetiersetablissement',
       'anneeeffectifsetablissement', 'caractereemployeuretablissement',
       'codecedex2etablissement', 'codecedexetablissement',
       'codecommune2etablissement', 'codecommuneetablissement',
       'codepaysetranger2etablissement', 'codepaysetrangeretablissement',
       'codepostal2etablissement', 'codepostaletablissement',
       'complementadresse2etablissement', 'complementadresseetablissement',
       'datecreationetablissement', 'datedebut',
       'datederniertraitementetablissement',
       'denominationusuelleetablissement',
       'distributionspeciale2etablissement',
       'distributionspecialeetablissement', 'enseigne1etablissement',
       'enseigne2etablissement', 'enseigne3etablissement',
       'etablissementsiege', 'etatadministratifetablissement',
       'indicerepetition2etablissement', 'indicerepetitionetablissement',
       'libellecedex2etablissement', 'libell

In [93]:
%%time
etab_df = (pd.read_csv(os.path.join(DATA_DIR, ETAB_FILENAME), # Concaténation du répertoire et du nom de fichier
                       usecols=['siren',
                                'activiteprincipaleetablissement', 'datecreationetablissement',
                                'enseigne1etablissement', 'denominationusuelleetablissement',
                                'etatadministratifetablissement',
                                'codepostaletablissement', 'codecommuneetablissement', 'libellecommuneetablissement',
                               ],
                           compression='zip',
                           #nrows=1_000_000 # limite à n lignes la lecture
                          )
             ).dropna(subset=['activiteprincipaleetablissement']).query('etatadministratifetablissement == "A"')

# Filtrage des professionnels de santé
#etab_df = etab_df[etab_df.activiteprincipaleetablissement.str.startswith(ACTIVITY_CODE)] # 86.

<string>:2: DtypeWarning: Columns (20,41,44) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 1min 28s


In [94]:
etab_df.shape

(11233968, 9)

In [80]:
etab_df.head(10)

,siren,datecreationetablissement,codepostaletablissement,libellecommuneetablissement,codecommuneetablissement,etatadministratifetablissement,enseigne1etablissement,denominationusuelleetablissement,activiteprincipaleetablissement
892,5720164,1983-09-28,80100.0,ABBEVILLE,80001,A,NaN,NaN,86.10Z
6424,7050040,1970-01-01,4510.0,AIGLUN,04001,A,NaN,NaN,86.10Z
17575,17150533,1971-01-01,21000.0,DIJON,21231,A,NaN,NaN,86.10Z
26233,24080749,2002-12-19,97600.0,MAMOUDZOU,97611,A,NaN,NaN,86.90B
34605,35204429,2009-01-01,31200.0,TOULOUSE,31555,A,NaN,NaN,86.23Z
37513,36220150,1997-01-27,6600.0,ANTIBES,06004,A,NaN,POLE ANTIBES SAINT JEAN,86.10Z
41402,37240348,1985-01-01,39100.0,DOLE,39198,A,NaN,NaN,86.90B
41851,37739802,2004-10-01,64100.0,BAYONNE,64102,A,NaN,NaN,86.21Z
112251,46620241,1966-01-01,20117.0,OCANA,2A181,A,NaN,NaN,86.10Z
112800,46750147,1967-01-01,59500.0,DOUAI,59178,A,NaN,NaN,86.10Z


In [81]:
etab_df.dtypes

siren                                 int64
datecreationetablissement            object
codepostaletablissement             float64
libellecommuneetablissement          object
codecommuneetablissement             object
etatadministratifetablissement       object
enseigne1etablissement               object
denominationusuelleetablissement     object
activiteprincipaleetablissement      object
dtype: object

In [82]:
entities_df.dtypes

siren                             int64
datecreationunitelegale          object
etatadministratifunitelegale     object
nomunitelegale                   object
denominationunitelegale          object
activiteprincipaleunitelegale    object
dtype: object

In [83]:
process_df = etab_df.merge(entities_df, on='siren', how='left')

In [84]:
process_df.shape

(560103, 14)

In [86]:
process_df[process_df.activiteprincipaleunitelegale.isnull()].shape

(7954, 14)

In [105]:
etab_df.memory_usage(deep=True).sum() / (1024 * 1024) # En mégaoctet

4456.694919586182

In [106]:
entities_df.memory_usage(deep=True).sum() / (1024 * 1024) # En mégaoctet

3464.3372735977173

In [107]:
etab_df.merge(entities_df, on='siren', how='outer', indicator=True)['_merge'].value_counts()

both          11233945
right_only     1622950
left_only           23
Name: _merge, dtype: int64